In [ ]:
!pip install qsprpred

In [ ]:
!pip install scikit-learn==1.6.1

In [ ]:
!pip install numpy==2.0.2

In [ ]:
!pip install shap

## <mark>Instructions</mark>
1. restart the session
2. download [QSPRPred data file](https://onedrive.live.com/?redeem=aHR0cHM6Ly8xZHJ2Lm1zL3UvcyFBdHpXcXUwaW5ralgzUVJ4WE9rVEZOdjdJVjd1P2U9UFBqME8y&cid=D7489E22EDAAD6DC&id=D7489E22EDAAD6DC%2111908&parId=D7489E22EDAAD6DC%21107&o=OneUp)
2. upload QSPRPred data file to current working directory
3. unzip it into `tutorial_data`




In [ ]:
%matplotlib inline

import os
# import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import shap
from IPython.display import display

# data preparation
from qsprpred.data import QSPRDataset
from qsprpred.data import RandomSplit, ClusterSplit
from qsprpred.data.descriptors.fingerprints import MorganFP, MaccsFP

# model preparation
import sklearn
# from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.stats import spearmanr

import xgboost
# from qsprpred.models import SklearnModel
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from qsprpred.models import CrossValAssessor, TestSetAssessor

from exai_tutorial import get_A2A_data, get_esol_data
from exai_tutorial import get_smarts, get_smarts_by_number
from exai_tutorial import score_model, feature_removal_score

In [ ]:
data_dir = "tutorial_data"
out_dir = "tutorial_output"
out_data_dir = os.path.join(out_dir, "data")
models_dir = os.path.join(out_dir, "models")

os.makedirs(out_data_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

# Task: classification
## Data preparation

In [ ]:
dataset = get_A2A_data(data_dir, out_data_dir)
display(dataset.getDF())

<mark>Choose which split to use</mark>

In [ ]:
# Specifiy random split for creating the train (80%) and test set (20%)
rand_split = RandomSplit(test_fraction=0.2, dataset=dataset)
clst_split = ClusterSplit(test_fraction=0.2, dataset=dataset)

# calculate compound features and split dataset into train and test
dataset.prepareDataset(
    split=rand_split,
    # split=clst_split,
    feature_calculators=[MaccsFP()],
)
dataset.save()

In [ ]:
print(f'train set size: {len(dataset.X)}')
print(f'test set size: {len(dataset.X_ind)}')

## Training the model
We'll train an XGBoost model.

In [ ]:
xgb_model = xgboost.XGBClassifier()
_ = xgb_model.fit(dataset.X, dataset.y)

acc_test = accuracy_score(xgb_model.predict(dataset.X_ind), dataset.y_ind)
roc_test = roc_auc_score(xgb_model.predict(dataset.X_ind), dataset.y_ind)

print(f'test set accuracy: {acc_test:.2f}')
print(f'test set roc: {roc_test:.2f}')

## Calculating SHAP values

<mark>First, we must define an explainer. Try figuring out what parameter values to use.</mark>

You can use the [documentation](https://shap.readthedocs.io/en/latest/generated/shap.TreeExplainer.html#shap.TreeExplainer)

In [ ]:
# define explainer
xgb_explainer = shap.TreeExplainer('your code here')

<mark>Calculate SHAP values on train and test set</mark>

In [ ]:
sv_train = 'your code here'
sv_test = 'your code here'

### <mark>**Ex. 0:**</mark> Feature MACCS_0 is always set to 0, is it important?
You can use `np.sum` function.

In [ ]:
shap_sum = 'your code here'  # calculate a sum of SHAP values for each sample
print(shap_sum[0], shap_sum[0]/sv_train.values.shape[0])

**conclusion:**

#### <mark>**Ex. 1:**</mark> Check whether SHAP values sum to the prediction.

- Use `model.predict_proba()`.
- `sv_train.base_values` holds base value.

In [ ]:
preds = 'your code here'  # calculate predictions
diff = 'your code here'   # calculate difference between predictions and summed SHAP values

print(min(diff), max(diff), np.var(diff))

**conclusion:**

#### <mark>**Ex. 2:**</mark> Gini coefficients and SHAP values are two ways of explaning tree-based models. Both methods are meant to estimate the importance of input features; however, while Gini coefficient is an impurity-based measure, SHAP values are formulated in such a way to ensure fair attribution.
#### Do you think Gini and SHAP will provide the same results? Analyse it in cells below.

First, we calculate and visualise Gini coefficients.

In [ ]:
gini = xgb_model.feature_importances_  # The impurity-based feature importances

In [ ]:
plt.figure()
plt.scatter(gini, np.absolute(sv_train.values).sum(axis=0))
plt.show()

<mark>Compare Gini and SHAP</mark>

In [ ]:
# Correlations of -1 or +1 imply an exact linear relationship. Positive correlations imply that as x increases, so does y
'your code here'

**conclusion:**

In [ ]:
'your code here'

**conclusion:**

In [ ]:
'your code here'

**conclusion:**

<mark>Which feature was ranked most different by SHAP and Gini?</mark>

- Remember that function `np.argsort` sorts from lowest to largest values.
- `np.where` might be helpful
- `np.argmax` might also be helpful

[smarts.plus](https://smarts.plus/) can be used to visualise smarts patterns

In [ ]:
gini_rank = 'your code here'  # calculate rank
shap_rank = 'your code here'  # calculate rank

rank_diff = 'your code here'  # calculate rank difference

diffest_feature = 'your code here'  # calculate index of the feature for which rank difference is the biggest
distance = rank_diff[diffest_feature]

print(f'feature name: {sv_train.feature_names[diffest_feature]}, feature SMARTS: {get_smarts_by_number(diffest_feature)}; distance: {distance}')

**conclusion:**

<mark>If we chose k most important features using Gini and using SHAP, would we get the same features?</mark>

In [ ]:
for k in[5, 10, 25, 50]:
  gini_k = gini_rank[:k]
  shap_k = shap_rank[:k]
  n_common = len(set(gini_k).intersection(set(shap_k)))
  print(f'for k={k} the intersection ratio is {n_common/k}')

**conclusion:**

#### <mark>**Ex. 3:**</mark> Visualisation of SHAP explanations

The SHAP library offers a range of different visualisations. Let's see some of them!

In [ ]:
# sample a random molecule from the dataset
sample_id = 100
sample_id = np.random.randint(len(sv_train))
print(sample_id)

In [ ]:
plt.figure()
shap.plots.decision(sv_train.base_values[sample_id], sv_train.values[sample_id, :], feature_names=sv_train.feature_names, feature_display_range=slice(-1, -10, -1))


In [ ]:
plt.figure()
shap.plots.waterfall(sv_train[sample_id], max_display=5, show=True)

In [ ]:
shap.plots.initjs()  # this is important for forceplots and can be done only once
plt.figure()
shap.plots.force(sv_train[sample_id])

#### <mark>**Ex. 4:**</mark> Analyse relationship between SHAP values and prediction change

In [ ]:
most_important_idx = np.argmax(np.absolute(sv_train[sample_id].values))
print('highest SHAP value', sv_train.values[sample_id, most_important_idx])

sample = sv_train[sample_id].data
changed_sample = sample.copy()
changed_sample[most_important_idx] = not changed_sample[most_important_idx]

pred = xgb_model.predict_proba(sample.reshape(1, -1))[0]  # [0] - 'cause nested arrays attack
changed_pred = xgb_model.predict_proba(changed_sample.reshape(1, -1))[0]
print('prediction change', (pred-changed_pred)[1])  # [1] - active class probability

**conclusion:**

# Let's switch to regression!

#### Download and prepare ESOL data

In [ ]:
esol = get_esol_data()
esol = QSPRDataset(name='ESOL', df=esol, store_dir=out_data_dir,
                    smiles_col='smiles',
                    target_props=[{"name":'ESOL predicted log solubility in mols per litre', "task":"regression"}],
                    overwrite=True,
                    random_state=42)


<mark>Choose which split to use.</mark>

In [ ]:
# Specifiy random split for creating the train (80%) and test set (20%)
erand_split = RandomSplit(test_fraction=0.2, dataset=dataset)
eclst_split = ClusterSplit(test_fraction=0.2, dataset=dataset)

# calculate compound features and split dataset into train and test
esol.prepareDataset(
    split=eclst_split,
    feature_calculators=[MaccsFP()],
)

esol.save()

## Train the model

In [ ]:
exgb = xgboost.XGBRFRegressor()
exgb = exgb.fit(esol.X, esol.y)

scores = score_model(xgb_model, dataset)
for key in sorted(scores):
  print(f'{key}: {scores[key]:.2f}')

## Explain

<mark>Define an explainer and calculate explanations on train and test set.</mark>

You can use [documentation](https://shap.readthedocs.io/en/latest/generated/shap.TreeExplainer.html#shap.TreeExplainer).

In [ ]:
regsplainer = shap.TreeExplainer('your code here')
essv_tr = 'your code here'
essv_te = 'your code here'

In [ ]:
# calculating gini
esgini = exgb.feature_importances_

#### <mark>**Ex. 1:**</mark> Check whether SHAP values sum to the prediction.

Use `model.predict()`.

In [ ]:
es_preds = 'your code here'     # calculate predictions
es_shap_sum = 'your code here'  # calculate the sum of SHAP values for each sample

In [ ]:
es_diff = 'your code here'  # calculate the difference between predictions and summed SHAP values
print(es_diff.min(), es_diff.max(), np.mean(es_diff), np.var(es_diff))

**conclusion:**

#### <mark>**Ex. 2:**</mark> Compare Gini coefficients and SHAP values

In [ ]:
# Correlations of -1 or +1 imply an exact linear relationship. Positive correlations imply that as x increases, so does y
'your code here'

**conclusion:**

<mark>Which feature was ranked most different by SHAP and Gini?</mark>

Remember that function `np.argsort` sorts from lowest to largest values.

In [ ]:
e_gini_rank = 'your code here'  # calculate rank
e_shap_rank = 'your code here'

rank_diff = 'your code here'  # calculate the difference between rankings

diffest_feature = np.argmax(rank_diff)
distance = rank_diff[diffest_feature]

print(f'feature name: {essv_tr.feature_names[diffest_feature]}, feature SMARTS: {get_smarts_by_number(diffest_feature)}; distance: {distance}')

<mark>If we chose k most important features using Gini and using SHAP, would we get the same features?</mark>

In [ ]:
# if we chose k most important features using Gini and using SHAP, would we get the same features?
for k in[5, 10, 25, 50]:
  gini_k = e_gini_rank[:k]
  shap_k = e_shap_rank[:k]
  n_common = len(set(gini_k).intersection(set(shap_k)))
  print(k, n_common/k)

**conclusion:**

#### <mark>**Ex. 3:**</mark> Visualisation of SHAP explanations

In [ ]:
sample_id = np.random.randint(len(esol.X))
print(sample_id)

In [ ]:
plt.figure()
shap.plots.waterfall(essv_tr[sample_id])

#### <mark>**Ex. 4:**</mark> Analyse relationship between SHAP values and prediction change

In [ ]:
most_important_idx = np.argmax(np.absolute(essv_tr[sample_id].values))
sample = essv_tr[sample_id].data
changed_sample = sample.copy()
changed_sample[most_important_idx] = not changed_sample[most_important_idx]
print(f'highest SHAP value: {essv_tr.values[sample_id, most_important_idx]}')

pred = exgb.predict(sample.reshape(1, -1))[0]  # [0] - 'cause nested arrays attack
changed_pred = exgb.predict(changed_sample.reshape(1, -1))[0]
print('prediction change:', (pred-changed_pred))

**conclusion:**

## Let's play around some more!

<mark>Select your favorite MACCS key</mark>

In [ ]:
favourite = 68
get_smarts(f'MACCSFP_MACCSFP_{favourite}'), get_smarts_by_number(favourite), get_smarts_by_number([145, favourite])

Let's see what molecules in the dataset contain this pattern

In [ ]:
df_smarts = esol.searchWithSMARTS([get_smarts_by_number(favourite)[0]])
display(df_smarts.df)

# Model debugging
#### <mark>**Ex. 5:**</mark> Compare which features are most important among correctly and incorrectly predicted samples.

analyse correct predictions:

In [ ]:
preds = 'your code here'  # calculate predictions
true = 'your code here'   # true labels
errors = np.absolute(preds-true)

In [ ]:
correct_threshold = 1
correct_mask = errors<correct_threshold

correct_ratio = np.sum(correct_mask)/len(errors)
print(f'Ratio of correctly predicted samples: {correct_ratio}')

In [ ]:
correct_preds=preds[correct_mask]
correct_shaps = essv_tr[correct_mask]

In [ ]:
# which features are the most important?
plt.figure()
shap.plots.bar(correct_shaps)

In [ ]:
print(f'the most important feature is: {get_smarts_by_number('your code here')}')

In [ ]:
# find 10 most important features
global_feature_importance = np.mean(np.absolute(correct_shaps.values), axis=0)
k = 10
top_k = np.flip(np.argsort(global_feature_importance)[-k:])
print(f'TOP{k} important features:')
for rank_position, sm in enumerate(get_smarts_by_number(top_k)):
  print(f'{rank_position+1}. {sm}')

analyse incorrect predictions

In [ ]:
# what about incorrect?
incorrect_preds=preds[np.invert(correct_mask)]
incorrect_shaps = essv_tr[np.invert(correct_mask)]

inc_global_feature_importance = np.mean(np.absolute(incorrect_shaps.values), axis=0)
inc_top_k = np.flip(np.argsort(inc_global_feature_importance)[-k:])

print(f'TOP{k} important features:')
for rank_position, sm in enumerate(get_smarts_by_number(inc_top_k)):
  print(f'{rank_position+1}. {sm}')

What is the difference between the most important features for correct and incorrect predictions?

In [ ]:
diff = set(top_k).symmetric_difference(set(inc_top_k))
print(f'features present in only one of the TOP{k} sets: {get_smarts_by_number(list(diff))}')

In [ ]:
print('  correct:', top_k)
print('incorrect:', inc_top_k)

Now, we can compare rankings as before to see which feature makes the biggest difference.

Then maybe we can train xgboost without this feature and see if it improves...

In [ ]:
# which feature was ranked most different when SHAP values were calculated on correct and incorrect samples?
corr_shap_rank = 'your code here'  # rank based on correct predictions
incr_shap_rank = 'your code here'  # rank based on incorrect predictions

rank_diff = 'your code here'  # difference between rankings

diffest_feature = np.argmax(rank_diff)
distance = rank_diff[diffest_feature]

print(f'feature name: {correct_shaps.feature_names[diffest_feature]}, feature SMARTS: {get_smarts_by_number(diffest_feature)}; distance: {distance}')

In [ ]:
print(f'Rank position among correct: {np.where(corr_shap_rank==diffest_feature)[0][0]}')
print(f'Rank position among incorrect: {np.where(incr_shap_rank==diffest_feature)[0][0]}')

Which molecules contain this substructure (this table also has information whether the molecules are in train or test)

In [ ]:
display(esol.searchWithSMARTS([get_smarts_by_number(diffest_feature)[0]]).df)

Was this feature on average more important for correct or incorrect predictions? Can we come up with an idea why the model makes mistakes (chemical interpretation - is this feature important for solubility etc.)? Can we propose a solution?

#### <mark>**Ex. 6:**</mark> Performance drop
Now, we can use the rankings to train a model without the most important features and see if the performance drops.

Less features => less information; therefore, for comparison we should also remove 5 random features.


In [ ]:
scores_orig = score_model(exgb, esol)
scores_orig.update(k=0, method='original')
scores = [scores_orig]
for k in [5, 10, 15]:
  # `feature_removal_score` removes features according to different scenarios
  # and then trains and evaluates a new model
  # only scores are returned
  scores.extend(feature_removal_score(k, esol, correct_shaps, corr_shap_rank))

scores_df = pd.DataFrame.from_records(scores)
display(scores_df)

Let's visualise the results

In [ ]:
_ = sns.scatterplot(data=scores_df, x='k', y='test_mse', hue='method')
plt.show()

In [ ]:
_ = sns.scatterplot(data=scores_df, x='k', y='test_r2', hue='method')
plt.show()

**conclusion:**

#### <mark>**Ex. 7:**</mark> Interaction values
SHAP allows to calculate and analyse interaction values.

In [ ]:
interaction_values = regsplainer.shap_interaction_values(esol.X)  # takes 10 minutes

In [ ]:
print(f'Indices of TOP{k} most important features {top_k}')

In [ ]:
# checking feature against itself
shap.dependence_plot(
    (145, 145),
    interaction_values,
    esol.X,
    display_features=esol.X,
)

In [ ]:
shap.dependence_plot(
    (145, 128),
    interaction_values,
    esol.X,
    display_features=esol.X,
)

**conclusion:**

In [ ]:
get_smarts_by_number([145, 128])

## <mark>**Homework**</mark>
**Thesis:** In structurally analogous compounds, same features are indicated as important (there is a consistency of ranking of individual features).

Find similar compounds in the datasets and compare their SHAP-based feature rankings.

Hint: *Interpretation of machine learning models using shapley values: application to compound potency and multi‐target activity predictions* by R. Rodríguez-Pérez and J. Bajorath
